In [1]:
import re
import pickle
import json
import pandas as pd

from library.common import Core
from pathlib import Path
cwd = Path.cwd()
prediction_horizon = 5
core = Core()
models = core.all_features

In [2]:
models_path = cwd.parent/'models'
models = list(models_path.glob('*.pkl'))

In [3]:
root_dir = cwd.parent
data_file= r'data/processed/params_forecast_params.json'
last_data_year_file = root_dir/data_file
file_path = last_data_year_file.as_posix()

In [4]:

with open(file_path, 'r') as datapoint:
    last_data_year = json.load(datapoint)

year_labels = [n for n in range(last_data_year.get('last_period') + 1,
                                last_data_year.get('last_period') + 1 +
                                prediction_horizon, 1)]
print('FORECAST LABELS: ', year_labels)

FORECAST LABELS:  [2021, 2022, 2023, 2024, 2025]


In [5]:
# regex = r"-([^-]\w*[^-])-([^-]\w*[^\.])"
regex = r"model-([a-zA-Z0-9|\s]*)-([\w\d]*_{0,1}[\w\d]*)"

forecasts = []
for model in models:
    # print('Processing:', model)
    fname = model.name
    matches = re.search(regex, fname)

    if matches:
        region = matches.group(1)
        feature = matches.group(2)

        print(region, feature)
        print(model.as_posix())

        with open(model.as_posix(), 'rb') as pkl:
            optimal = pickle.load(pkl)

            f = optimal.predict(prediction_horizon)
            print('Forecast ', f)
            fc = zip(year_labels, f)
            forecast_df = pd.DataFrame(fc)
            forecast_df.columns = ['year', 'forecasts']
            forecasts.append([region, feature, forecast_df.to_numpy()])


Cluster 0 trade_openness
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-Cluster 0-trade_openness.pkl
Forecast  [4399.42991989 4399.42991989 4399.42991989 4399.42991989 4399.42991989]
Cluster 2 manufacturing_percent
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-Cluster 2-manufacturing_percent.pkl
Forecast  [1035.57626875 1027.38277684 1019.18928492 1010.99579301 1002.80230109]
Cluster 2 co2
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-Cluster 2-co2.pkl
Forecast  [16736.47064652 16643.82962381 16578.09211072 16516.86187941
 16456.38677594]
Global population
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/model-Global-population.pkl
Forecast  [7.83405760e+09 7.91361755e+09 7.99317749e+09 8.07273744e+09
 8.15229738e+09]
Cluster 0 constant_gdp_per_capita
/Users/koonleong/Documen

In [6]:
fc_path = cwd.parent/r'data/processed/ts_forecast.pkl'
with open(fc_path, 'wb') as pkl:
    pickle.dump(forecasts, pkl)

print("Done with Forecasts")

Done with Forecasts


In [7]:
# Code to read the forecast
from pathlib import Path
cwd = Path.cwd()
fc_path = cwd.parent/r'data/processed/ts_forecast.pkl'
with open(fc_path, 'rb') as fc_file:
    read_fc = pickle.load(fc_file)
print(read_fc)

[['Cluster 0', 'trade_openness', array([[2021.        , 4399.42991989],
       [2022.        , 4399.42991989],
       [2023.        , 4399.42991989],
       [2024.        , 4399.42991989],
       [2025.        , 4399.42991989]])], ['Cluster 2', 'manufacturing_percent', array([[2021.        , 1035.57626875],
       [2022.        , 1027.38277684],
       [2023.        , 1019.18928492],
       [2024.        , 1010.99579301],
       [2025.        , 1002.80230109]])], ['Cluster 2', 'co2', array([[ 2021.        , 16736.47064652],
       [ 2022.        , 16643.82962381],
       [ 2023.        , 16578.09211072],
       [ 2024.        , 16516.86187941],
       [ 2025.        , 16456.38677594]])], ['Global', 'population', array([[2.02100000e+03, 7.83405760e+09],
       [2.02200000e+03, 7.91361755e+09],
       [2.02300000e+03, 7.99317749e+09],
       [2.02400000e+03, 8.07273744e+09],
       [2.02500000e+03, 8.15229738e+09]])], ['Cluster 0', 'constant_gdp_per_capita', array([[  2021.        , 6573